In [1]:
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

#change directory
%cd ..

/home/jovyan/inference_project/preprocess_mimic_update


In [2]:
### parameters ###

k = 5 # number of top codes to display

In [3]:
def result_log(df, line):
    print('Text:')
    pp.pprint(df.loc[line]["text"])
    print('Code predicted:')
    print(df.loc[line]["top1_column"])
    print('Confidence:')
    print(df.loc[line]["top1_value"])

def select_top_k(k, df):
    result_df = df[['_id', 'target']]
    result_df.reset_index(drop = True, inplace = True)
    
    df_values = df.loc[:, ~df.columns.isin(['_id', 'target'])]
    
    result_df_values = pd.DataFrame()
    #create columns 
    for i in range(k):
        result_df_values['top{}_column'.format(i+1)] = None
        result_df_values['top{}_value'.format(i+1)] = None
    
    #set values to the new columns
    for i  in range(len(df_values)):
        top_k = df_values.iloc[i].sort_values(ascending= False)[:k]
        keys = top_k.index
        values = top_k.values
        
        keys_values = [item for pair in zip(keys, values) for item in pair]
    
        result_df_values.loc[i] = keys_values
        
    result_df = pd.concat([result_df, result_df_values], axis=1)

    return result_df

# Analyse MIMIC ICD

## Macro

In [4]:
mimic_axa_data = pd.read_feather('files/data/mimic_axa_icd/mimic_axa_icd10.feather')

In [5]:
test_results = pd.read_feather('files/mimic_axa/predictions_test.feather')
test_results = test_results[-500:] #select last 500 rows
test_results_top_k = select_top_k(k=k, df = test_results)
test_results_top_k = test_results_top_k.merge(mimic_axa_data[['_id', 'text']], on = '_id')
test_results_top_k

,_id,target,top1_column,top1_value,top2_column,top2_value,top3_column,top3_value,top4_column,top4_value,top5_column,top5_value,text
0,26576572,"[J44.1, Z79.02, E87.2, J96.92, E87.1, E66.9, G...",G47.33,0.957063,I10.,0.907808,E78.5,0.879294,K21.9,0.854116,J44.1,0.841102,name unit no admission date discharge date dat...
1,26860865,"[E87.2, S72.141A, K83.8, E87.1, I25.10, E78.5,...",Z95.1,0.969391,E11.65,0.965311,S72.141A,0.926209,I25.10,0.908887,E78.5,0.906897,name unit no admission date discharge date dat...
2,21990791,"[J18.9, F10.21, K65.2, E87.1, Y92.230, B19.20,...",K70.31,0.974383,K76.7,0.962007,I81.,0.954034,E43.,0.867934,I25.10,0.844359,name unit no admission date discharge date dat...
3,23999593,"[R58., I25.10, K76.6, N18.3, B96.20, D62., D69...",N18.3,0.982522,Z95.1,0.962391,K21.9,0.954747,N17.9,0.931586,I25.10,0.911157,name unit no admission date discharge date dat...
4,21485131,"[Z85.79, K75.81, I25.10, K76.6, E78.5, R50.9, ...",N18.2,0.950203,F32.9,0.947790,I81.,0.932105,K75.81,0.903965,I25.10,0.897869,name unit no admission date discharge date dat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,25168203,"[I48.0, I82.421, K65.9, R31.9, K57.32, E66.9, ...",I25.10,0.965500,I26.99,0.927452,Z95.5,0.922959,I82.431,0.906833,K91.89,0.896912,name unit no admission date discharge date dat...
496,20094862,"[I67.6, E79.0, R73.9, H57.02, E66.9, C90.00, Y...",E43.,0.983732,Z86.711,0.969907,N17.9,0.965293,C90.00,0.950297,K59.00,0.943877,name unit no admission date discharge date dat...
497,22429647,"[E87.1, E46., D68.4, Z78.1, D62., Z90.49, I48....",Z79.01,0.933163,Z79.4,0.886894,I48.91,0.864890,K91.89,0.850498,R65.21,0.818824,name unit no admission date discharge date dat...
498,25077908,"[E87.2, I42.8, A41.89, I82.622, I07.1, J80., E...",I42.0,0.866141,E03.9,0.861080,I82.622,0.827124,B37.0,0.805016,Z95.811,0.761795,name unit no admission date discharge date dat...


## Examples

In [10]:
result_log(test_results_top_k, 1)

Text:
('name unit no admission date discharge date date of birth sex m service '
 'medicine allergies no known allergies adverse drug reactions attending chief '
 'complaint right hip pain major surgical or invasive procedure right hip tfn '
 'ercp with sphincterotomy flexible sigmoidoscopy history of present illness '
 'yo speaking male with hld dm and cad s p cabg who was transferred from to on '
 'after a mvc found to have a r intertrochanteric fracture planned for orif '
 'with orthopedics now transferred to medicine due to difficult to control '
 'blood sugars hyponatremia hyperkalemia and leukocytosis pt was a restrained '
 'driver with no loc or headstrike during the above noted mvc and patient was '
 'taken to for evaluation vs were t97 hr bp rr o2sat exam was notable for '
 'diffuse pain with rom at the r hip with mild shortening mild tenderness to '
 'palpation over paraspinal muscles but no midline tenderness and open skin '
 'abrasions over dorsum of the r hand labs at nota

# Analyse MIMIC-AXA ICD

## Macro

In [8]:
axa_data = pd.read_feather('files/data/axa_icd10/axa_icd10.feather')
ids_axa = list(axa_data['_id'])

In [9]:
val_results = pd.read_feather('files/mimic_axa/predictions_val.feather')
val_results_axa = val_results[val_results['_id'].isin(ids_axa)]
val_results_axa_top_k = select_top_k(k=k, df = val_results_axa)
val_results_axa_top_k = val_results_axa_top_k.merge(axa_data[['_id', 'text']], on = '_id')
val_results_axa_top_k

,_id,target,top1_column,top1_value,top2_column,top2_value,top3_column,top3_value,top4_column,top4_value,top5_column,top5_value,text
0,13117700,[Z71.8],Z32.1,0.269132,Z71.8,0.049279,S93,0.037659,M51.1,0.022750,U07.1,0.022151,current disease cause of disease diagnostic tr...
1,13285005,[Z71.8],Z32.1,0.269132,Z71.8,0.049279,S93,0.037659,M51.1,0.022750,U07.1,0.022151,current disease cause of disease diagnostic tr...
2,13067682,[J93.1],Z32.1,0.175665,Z71.8,0.075976,S93,0.055621,S01,0.043133,U07.1,0.033669,current disease cause of disease diagnostic ex...
3,13230779,[A09],Z32.1,0.055442,Z71.8,0.045934,S93,0.035508,Z23.,0.026060,U07.1,0.018586,current disease cause of disease bacterial dia...
4,13430163,[M51.1],M51.1,0.708517,M48.061,0.064162,Z71.8,0.058990,Z32.1,0.057656,S93,0.047867,current disease post operated lumbar instrumen...
5,13199485,[Z32.1],Z32.1,0.524967,Z71.8,0.066648,S93,0.057682,Z23.,0.029644,M51.1,0.029338,current disease i prevent post resumer electiv...
6,12935394,[Z32.1],Z32.1,0.859061,S93,0.109357,Z71.8,0.088751,M51.1,0.055600,S01,0.032615,current disease secondary amenorhea due to pre...
7,13100091,[Z32.1],Z32.1,0.807568,S93,0.058492,Z71.8,0.055710,Z23.,0.034882,U07.1,0.024374,current disease di de bicemal biar twin pregna...
8,13224756,[Q40.1],Z32.1,0.125357,S93,0.092911,Z71.8,0.078219,K21.9,0.061460,K44.9,0.059905,current disease patient with presence of parae...
9,13386965,[Z32.1],Z32.1,0.816483,Z71.8,0.046660,S93,0.041094,Z23.,0.021045,U07.1,0.017104,current disease without uterine activity witho...


## Examples

In [38]:
result_log(val_results_axa_top_k, 45)

Text:
('current disease ailment the patient comes to vyveces par having presented '
 'care with a truma in the left hand for which he started candola and '
 'imitation e f edens it equiiny functional hand dollar to the pulp without '
 'distal neurocinelatin compromise ailment the patient comes to vyveces par '
 'having presented care with a truma in the left hand for which he started '
 'candola and imitation e f edens it equiiny functional hand dollar to the '
 'pulp without distal neurocinelatin compromise ailment the patient comes to '
 'vyveces par having presented care with a truma in the left hand for which he '
 'started candola and imitation e f edens it equiiny functional hand dollar to '
 'the pulp without distal neurocinelatin compromise cause of disease time '
 'manner and place where the injury occurred loss of full sustenance it is '
 'related to another condition yestime manner and place where the injury '
 'occurred loss of full sustenance it is related to another condi

In [39]:
result_log(val_results_axa_top_k, 37)

Text:
('current disease it began in with hypoesthesia in the lower limbs and '
 'stabbing pain in the medial aspect of the right thigh a thoracic lumbar '
 'spine mri was performed finding a hyperintense spinal cord lesion on t2 '
 'sequence and stir at the level of the ninth and tenth thoracic vertebrae of '
 'demyelinating characteristics it was complemented with mri of the skull for '
 'periventricular lesions the diagnosis of multiple sclerosis was made cause '
 'of disease relapsing remitting multiple sclerosis diagnostic relapsing '
 'remitting multiple sclerosis pain in the left leg that limits walking '
 'persistent pain in the proximal and posterolateral region of the thigh and '
 'leg of a neuropathic radicular type treatment teriflunomide mg dia')
Code predicted:
G35.
Confidence:
0.88612944


In [40]:
result_log(val_results_axa_top_k, 38)

Text:
('current disease a year old male patient presents docon casteldia aby dominac '
 'burgclismos pyrosis liquid evaluations with the presence of sonore despite '
 'having attended to the general physician clausultation there is no mejuria '
 'he is a prisoner20 year old male patient commonly epigastric mesogastrium '
 'borgmismotireregurgitations pyrosis docor vente in cord colic acute dock tor '
 'distensind abdul inac with nocardiac ref with severe gastroesophageal '
 'causing nocturnal hitogos of mocta mooscopy with cos findings it is '
 'determined cause of disease acode to castroein outpatient office te mologia '
 'uonte la situation of fish symptomatology se medimca endoscopy cooking '
 'diagnostic ci therapeutic diagnostic itiatc hernia type severe cardiofiatal '
 'incontinence gastrito erojive iemorrhagic erosive duodenitis seanexan images '
 'treatment')
Code predicted:
K29.80
Confidence:
0.5020244


# Analyse MIMIC CPT

## Macro

In [11]:
mimic_axa_data = pd.read_feather('files/data/mimic_axa_cpt/mimic_axa_cpt.feather')

In [12]:
test_results = pd.read_feather('files/mimic_axa_cpt_hierarchical/predictions_test.feather')
test_results = test_results[-500:] #select last 500 rows
test_results_top_k = select_top_k(k=k, df = test_results)
test_results_top_k = test_results_top_k.merge(mimic_axa_data[['_id', 'text']], on = '_id')
test_results_top_k

,_id,target,top1_column,top1_value,top2_column,top2_value,top3_column,top3_value,top4_column,top4_value,top5_column,top5_value,text
0,129338,"[99231, 99232, 99291, 99233, 99221]",99232,0.973660,99291,0.944055,99233,0.922849,99231,0.911779,99223,0.668971,admission date discharge date date of birth se...
1,108422,"[11044, 99254, 94003, 94002, 27295, 99253]",94003,0.620137,99254,0.596398,99232,0.376956,99253,0.371038,99255,0.349827,admission date discharge date date of birth se...
2,172168,"[99239, 99231, 99222, 99232, 99291, 99233]",99232,0.893174,90935,0.777798,99233,0.746591,99231,0.699452,99291,0.659050,admission date discharge date date of birth se...
3,156533,"[99232, 99292, 94003, 99291, 99233, 94002, 99238]",94003,0.966968,99233,0.839252,99291,0.782749,99232,0.750997,99238,0.446714,admission date discharge date date of birth se...
4,165822,"[99231, 99223, 99232, 99222, 99292, 36556, 940...",99291,0.983289,99292,0.770006,99233,0.712909,99232,0.695812,90945,0.670695,admission date discharge date date of birth se...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,150284,"[99223, 31600, 20930, 22585, 99221, 94002, 228...",94003,0.993591,99232,0.926824,99233,0.895946,99291,0.782108,94002,0.776003,admission date discharge date date of birth se...
496,138842,"[99238, 99291, 99233, 99232]",99291,0.896501,99233,0.894871,99232,0.729197,99239,0.712967,99238,0.211574,admission date discharge date date of birth se...
497,105769,"[99223, 94003, 99233, 99261, 94002]",94003,0.995305,31624,0.904868,99291,0.810990,99233,0.770073,99232,0.738737,admission date discharge date date of birth se...
498,130939,"[99223, 99263, 94002, 90935, 99231, 31622, 992...",94003,0.995072,99233,0.980586,99232,0.968420,99291,0.944594,94002,0.845728,admission date discharge date date of birth se...


## Examples

In [13]:
result_log(test_results_top_k, 1)

Text:
('admission date discharge date date of birth sex m service surgery allergies '
 'penicillins ivp dye iodine containing attending first name3 lf chief '
 'complaint fever chills left leg pain redness and swelling major surgical or '
 'invasive procedure left hip disarticulation diverting descending colostomy '
 'splenic flexure mobilization of the colon gastrostomy tube placement repair '
 'of incisional hernia debridement of subcutaneous tissue including muscle of '
 'the left pelvis and gluteus vac dressing ivc filter primary wound closure '
 'history of present illness yo male with history of rectal carcinoma who '
 'presents after a fall one week ago and brbpr now with fevers chills left leg '
 'pain redness and swelling past medical history rectal cancer s p resection w '
 'ileostomy s p ileostomy takedown bilateral knee arthroscopies s p ventral '
 'hernia repair social history married owns men s clothing store in location '
 'un family history noncontributory physical exam

# Analyse MIMIC AXA CPT

## Macro

In [13]:
axa_data = pd.read_feather('files/data/axa_cpt/axa_cpt.feather')
ids_axa = list(axa_data['_id'])

In [14]:
val_results = pd.read_feather('files/mimic_axa_cpt_hierarchical/predictions_val.feather')
val_results_axa = val_results[val_results['_id'].isin(ids_axa)]
val_results_axa_top_k = select_top_k(k=k, df = val_results_axa)
val_results_axa_top_k = val_results_axa_top_k.merge(axa_data[['_id', 'text']], on = '_id')
val_results_axa_top_k

,_id,target,top1_column,top1_value,top2_column,top2_value,top3_column,top3_value,top4_column,top4_value,top5_column,top5_value,text
0,13437099,[97010],99204,0.542491,97010,0.227312,99284,0.082261,99291,0.060866,99232,0.038395,cause of disease degenerative actual disease t...
1,13480889,[99204],94003,0.216795,99253,0.180926,99254,0.167006,11012,0.160366,94002,0.131133,cause of disease accident forced inversion of ...
2,13398568,[99204],99253,0.277833,25608,0.241016,25605,0.235559,25600,0.197673,94003,0.176286,cause of disease fall with impact on the left ...


In [15]:
test_results = pd.read_feather('files/mimic_axa_cpt_hierarchical/predictions_test.feather')
test_results_axa = test_results[test_results['_id'].isin(ids_axa)]
test_results_axa_top_k = select_top_k(k=k, df = test_results_axa)
test_results_axa_top_k = test_results_axa_top_k.merge(axa_data[['_id', 'text']], on = '_id')
test_results_axa_top_k

,_id,target,top1_column,top1_value,top2_column,top2_value,top3_column,top3_value,top4_column,top4_value,top5_column,top5_value,text
0,13100771,"[99284, 15852]",99204,0.541795,97010,0.228145,99284,0.082800,99291,0.057959,99232,0.036797,cause of disease wounded by a firearm projecti...
1,13021869,[99204],99204,0.543174,97010,0.230861,99284,0.083379,99291,0.057961,99232,0.037600,cause of disease uncontrolled primary hypothyr...
2,13127804,[99204],99204,0.307167,99253,0.133405,99254,0.122027,97010,0.113378,99231,0.076276,cause of disease indirect trauma to the left a...
3,13116727,"[97010, 99204]",99204,0.316768,97010,0.157504,99291,0.087305,99232,0.052821,99233,0.042024,cause of disease dopamine deficiency idiopathi...


## Examples

In [51]:
result_log(val_results_axa_top_k, 2)

Text:
('cause of disease fall with impact on the left thoracic limb fall with impact '
 'on the left thoracic limb fall with impact on the left thoracic limb fall '
 'with impact on the left thoracic limb actual disease a year old female '
 'patient was sent with the diagnosis of left distal radius fracture he refers '
 'that he does not present pain but does have limitation of mobility and '
 'muscular weakness fall with impact on the left thoracic limb with one week '
 'of evolution which was attended in the emergency due to a fracture in the '
 'distal radius and managed with closed reduction and placement of a '
 'fiberglass appliance palmar brachy and presenting pain with radiography in '
 'which an angulation of greater than degrees is observed fall with impact on '
 'the left thoracic extremity of one week of evolution that was attended in '
 'the emergency for fracture in the distal radius and managed with closed '
 'reduction and fiber appliance placement glass shield and pres

In [52]:
result_log(test_results_axa_top_k, 1)

Text:
('cause of disease uncontrolled primary hypothyroidism and impaired cardiac '
 'function sec to aortic stenosis actual disease anything diagnostic the '
 'patient reported that she started approximately weeks prior to the '
 'consultation with dyspnea on great exertion as well as pain in the right '
 'hemithorax asthenia adynamia and increased dyspnea were added to the picture '
 'until it was from small efforts right pleural effusion uncontrolled primary '
 'hypothyroidism aortic stenosis congestive heart failure treatment diagnostic '
 'therapeutic thoracocentesis medical speciality internal medicine')
Code predicted:
99204
Confidence:
0.5431741
